In [1]:
import pandas as pd
import numpy as np

In [2]:
restaurants = pd.read_excel('Data_Restaurants.xlsx')
recipes     = pd.read_excel('Final_Recipe_Cleaned.xlsx')
nutrition   = pd.read_excel('Food_Nutrition_Cleaned_Final.xlsx')

In [ ]:
restaurants = restaurants.drop(columns=['Zipcode'])            # dropping as 76% data in this field is missing
nutrition   = nutrition.drop(columns=['Choline'])              # dropping as 48% data is missing
# I am adding this as columns with really less dtaa will add nosie and it will drop the data quality for my model trianing. 

In [ ]:
# Fixing the opening and closing timmming in redstaurant dataset
restaurants['OpeningHour'] = pd.to_datetime(restaurants['OpeningTime'], errors='coerce').dt.hour
restaurants['ClosingHour'] = pd.to_datetime(restaurants['ClosingTime'], errors='coerce').dt.hour
opening  = restaurants['OpeningHour'].mode()[0]
closing = restaurants['ClosingHour'].mode()[0]
restaurants['OpeningHour'].fillna(opening, inplace=True)
restaurants['ClosingHour'].fillna(closing, inplace=True)
# Extracted opening and closing hour. Filling the time data in columns where it is null on the basis of average. This will help me to preserve the data and calculate how many hours each place is open in the future.

/var/folders/yv/f9fb76l15fb9d7249jmc3v2w0000gn/T/ipykernel_18709/1446268867.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  restaurants['OpeningHour'] = pd.to_datetime(restaurants['OpeningTime'], errors='coerce').dt.hour
/var/folders/yv/f9fb76l15fb9d7249jmc3v2w0000gn/T/ipykernel_18709/1446268867.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  restaurants['ClosingHour'] = pd.to_datetime(restaurants['ClosingTime'], errors='coerce').dt.hour
/var/folders/yv/f9fb76l15fb9d7249jmc3v2w0000gn/T/ipykernel_18709/1446268867.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace metho

In [ ]:
cap = recipes['Recipe Servings'].quantile(0.99)
recipes['Recipe Servings'] = np.where(recipes['Recipe Servings'] > cap, cap, recipes['Recipe Servings'])
recipes['Recipe Servings'].fillna(recipes['Recipe Servings'].median(), inplace=True)
recipes['Rating'].fillna(recipes['Rating'].median(), inplace=True)
# Any dtaa hving to much large number of reicpes gets pushed into 99th percentile, if there is empty serving we change it ot the medium. 
# also if any recipe is missing raitng i am adidng average raitng to it. 

/var/folders/yv/f9fb76l15fb9d7249jmc3v2w0000gn/T/ipykernel_18709/1029406099.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  recipes['Recipe Servings'].fillna(recipes['Recipe Servings'].median(), inplace=True)
/var/folders/yv/f9fb76l15fb9d7249jmc3v2w0000gn/T/ipykernel_18709/1029406099.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object

In [ ]:
for col in ['VitaminK', 'VitaminE']:
    nutrition[col].fillna(nutrition[col].median(), inplace=True)
# As per EDA , there ar emsing values for vitamin E and K, blank places get median vitamin value. 

/var/folders/yv/f9fb76l15fb9d7249jmc3v2w0000gn/T/ipykernel_18709/389002.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nutrition[col].fillna(nutrition[col].median(), inplace=True)


In [ ]:
price_map = {'$':1, '$$':2, '$$$':3, '$$$$':4}
restaurants['price_bin'] = restaurants['PriceRange'].map(price_map).fillna(0).astype(int)
restaurants['hours_open'] = (restaurants['ClosingHour'] - restaurants['OpeningHour']).clip(lower=0)
restaurants['cost_bin'] = pd.qcut(restaurants['AverageCostForTwo'], q=4, labels=[1,2,3,4]).astype(int)
# Added a price bin to tell which restuanrt is how mich expensive, hours open tells hoe long the restuanrant is open for. 


In [10]:
recipes = recipes.rename(columns={
    'Num Ingredients': 'num_ingredients',
    'Num Steps':       'num_steps'
})

In [ ]:
nutrition['energy_density'] = (
    nutrition['Calories'] /
    (nutrition['Protein'] + nutrition['FatTotal'] + nutrition['Carbohydrates'])
).replace([np.inf, -np.inf], np.nan).fillna(0)
# here i am calculating calories per gram. 

In [12]:
restaurants.to_csv('Restaurants_Featured.csv', index=False)
recipes.to_csv('Recipes_Featured.csv', index=False)
nutrition.to_csv('Nutrition_Featured.csv', index=False)

In [ ]:
restaurants = restaurants.drop(columns=[
    'Timings', 'OpeningHour', 'ClosingHour'])

In [17]:
recipes = recipes.drop(columns=[col for col in ['num_ingredients.1','num_steps.1'] if col in recipes.columns])